# From Simple LP to Farmer's Field (Min-Cost) — Step by Step

This notebook introduces optimization in **increasing complexity**, starting from a tiny linear program with a no-dependency solver and building up to a **farmer's field** problem with a **BQP** (binary quadratic) objective. Finally, we show how to model the simple LP in **PuLP** for an exact solution.

**Sections**
1. **Simplest LP (No dependencies)** — tiny 2-variable LP solved by brute-force grid search.
2. **Farmer's Field (Minimize Fertilizer Cost, No dependencies)** — linear penalties, randomized hill-climbing.
3. **Farmer's Field as BQP (No dependencies)** — add adjacency cost and solve with binary local search.
4. **PuLP exact model for the simple LP** — MILP formulation and solve with CBC.


## 1) Simplest LP — Grid Search (No Dependencies)
We solve the textbook LP:

\[ \max 6x_1 + 3x_2 \]
subject to
\[ x_1 + 2x_2 \le 40, \quad 4x_1 + 3x_2 \le 120, \quad x_1, x_2 \ge 0. \]

We'll do a coarse **grid search** over a box and evaluate the objective with hard constraint checks (still no external packages).

In [ ]:
def feasible(x1, x2):
    return (x1 >= 0 and x2 >= 0 and
            x1 + 2*x2 <= 40 + 1e-9 and
            4*x1 + 3*x2 <= 120 + 1e-9)

def objective(x1, x2):
    return 6*x1 + 3*x2

best = None
best_val = -1e18

step = 1.0  # adjust for finer search (e.g., 0.5)
for i in range(0, 121):
    x1 = i*step
    if x1 > 120: break
    for j in range(0, 121):
        x2 = j*step
        if not feasible(x1, x2):
            continue
        val = objective(x1, x2)
        if val > best_val:
            best_val = val
            best = (x1, x2)

print('Best (grid): x1, x2 =', best, ' objective =', best_val)

## 2) Farmer's Field — Minimize Fertilizer Cost (No Dependencies)
We now solve a mini planning problem: choose which fields to plant to **minimize fertilizer cost** while meeting an **acreage requirement** and respecting a **water cap**. We use soft penalties and a randomized hill-climber.


In [ ]:
from typing import List, Tuple
import random

# Example dataset (editable)
Cost    = [9, 7, 6, 8, 12, 5, 11, 4, 4, 10, 3, 6]   # fertilizer cost if planted
Acreage = [5, 6, 3, 4,  8, 3,  7, 2, 2,  6, 1, 4]   # acres
Water   = [9, 8, 5, 7, 12, 4, 10, 3, 2,  8, 2, 6]   # water units

AREA_REQ  = 25
WATER_CAP = 50

n = len(Cost)
assert len(Acreage)==n and len(Water)==n

PEN_REQ    = 1e3
PEN_CAP    = 1e3
PEN_BINARY = 1.0

def obj_linear(xx: List[float]) -> float:
    total_cost = sum(Cost[i]*xx[i] for i in range(n))
    total_area  = sum(Acreage[i]*xx[i] for i in range(n))
    total_water = sum(Water[i]*xx[i]   for i in range(n))
    viol = 0.0
    if total_area < AREA_REQ:
        viol += PEN_REQ * (AREA_REQ - total_area)
    if total_water > WATER_CAP:
        viol += PEN_CAP * (total_water - WATER_CAP)
    bin_pen = PEN_BINARY * sum(x*(1.0-x) for x in xx)
    return total_cost + viol + bin_pen

def solve_linear(iters=30000, seed=42):
    random.seed(seed)
    x = [random.random() for _ in range(n)]
    f = obj_linear(x)
    best_x, best_f = x[:], f
    for t in range(iters):
        y = x[:]
        i = random.randrange(n)
        y[i] = min(1.0, max(0.0, y[i] + random.uniform(-0.25, 0.25)))
        fy = obj_linear(y)
        if fy < f:
            x, f = y, fy
            if f < best_f:
                best_x, best_f = x[:], f
        elif random.random() < 0.001:
            x = [random.random() for _ in range(n)]
            f = obj_linear(x)
            if f < best_f:
                best_x, best_f = x[:], f
    return best_x, best_f

def summarize(x: List[float]):
    picks = [i for i,xi in enumerate(x) if xi>=0.5]
    area = sum(Acreage[i] for i in picks)
    water= sum(Water[i]   for i in picks)
    cost = sum(Cost[i]    for i in picks)
    return picks, area, water, cost

x_lin, f_lin = solve_linear()
picks, area, water, cost = summarize(x_lin)
print('Dependency-free solution (approx):')
print('  chosen fields:', picks)
print('  acres:', area, '| water:', water, '| fertilizer cost:', cost)
print('  penalized objective:', round(f_lin, 3))

## 3) Farmer's Field as BQP (Adjacency Cost, No Dependencies)
We add a **quadratic adjacency cost**: planting two neighboring fields simultaneously incurs `ADJ_COST`.
We then use a **binary local search** with restarts.


In [ ]:
from typing import List, Tuple
import random

AdjacencyPairs: List[Tuple[int,int]] = [
    (0,1), (1,2), (1,3), (3,4), (4,6), (5,6),
    (6,7), (7,8), (8,9), (2,5), (9,10), (9,11)
]
ADJ_COST = 5.0

def obj_bqp(xx: List[int]) -> float:
    total_cost = sum(Cost[i]*xx[i] for i in range(n))
    adj = 0.0
    for (i,j) in AdjacencyPairs:
        adj += ADJ_COST * (xx[i] * xx[j])
    total_area  = sum(Acreage[i]*xx[i] for i in range(n))
    total_water = sum(Water[i]*xx[i]   for i in range(n))
    viol = 0.0
    if total_area < AREA_REQ:
        viol += 1e3 * (AREA_REQ - total_area)
    if total_water > WATER_CAP:
        viol += 1e3 * (total_water - WATER_CAP)
    return total_cost + adj + viol

def binary_local_search(iters=5000, restarts=40, seed=123):
    random.seed(seed)
    best_x, best_f = None, float('inf')
    for r in range(restarts):
        x = [1 if random.random()<0.5 else 0 for _ in range(n)]
        f = obj_bqp(x)
        improved = True
        steps = 0
        while improved and steps < iters:
            improved = False
            steps += 1
            idx = list(range(n))
            random.shuffle(idx)
            for i in idx:
                x[i] ^= 1
                f_new = obj_bqp(x)
                if f_new + 1e-9 < f:
                    f = f_new
                    improved = True
                else:
                    x[i] ^= 1
            if not improved and random.random() < 0.02:
                i = random.randrange(n)
                x[i] ^= 1
                f = obj_bqp(x)
                improved = True
        if f < best_f:
            best_x, best_f = x[:], f
    return best_x, best_f

x_bqp, f_bqp = binary_local_search()
picks_b = [i for i,xi in enumerate(x_bqp) if xi==1]
area_b  = sum(Acreage[i] for i in picks_b)
water_b = sum(Water[i]   for i in picks_b)
cost_b  = sum(Cost[i]    for i in picks_b)
print('BQP solution (approx):')
print('  chosen fields:', picks_b)
print('  acres:', area_b, '| water:', water_b, '| fertilizer cost:', cost_b)
print('  penalized objective:', round(f_bqp, 3))

## 4) PuLP — Exact Solve for the Simple LP
Here we formulate the **simple 2-variable LP** in PuLP and solve it exactly with CBC. If PuLP isn't installed, uncomment the install cell first.

In [ ]:
# If PuLP is not installed, uncomment:
# !pip install pulp

In [ ]:
try:
    import pulp
except ImportError:
    raise RuntimeError('PuLP is not installed. Run the install cell above or install in your environment.')

# Variables: x1, x2 >= 0
x1 = pulp.LpVariable('x1', lowBound=0)
x2 = pulp.LpVariable('x2', lowBound=0)

# Problem: maximize 6x1 + 3x2
prob = pulp.LpProblem('SimpleLP', pulp.LpMaximize)
prob += 6*x1 + 3*x2

# Constraints
prob += x1 + 2*x2 <= 40
prob += 4*x1 + 3*x2 <= 120

prob.solve(pulp.PULP_CBC_CMD(msg=False))
print('Status:', pulp.LpStatus[prob.status])
print('x1 =', x1.value())
print('x2 =', x2.value())
print('Objective =', pulp.value(prob.objective))